# Recipes Generator 
### Genetic Algorithm

Given an amount of __calories__, the model will provide several __recipes__ that fit the amount given. Using a data base stored in a csv file, this model select random samples based on the 'n_genes' number provided by the user.


In this model I implement a roulette selection and a method to create the population, this method consists in pick a random point between 1 and (n_cromosomas) - 1 to split the father and mother and make mix, generating two samples. Note: If you want, you can modify this parameter to make it static instead of stochastic.


In addition, I implement a custom method to handle overfitting or underfitting, the details are in the code below.


You can play with the next parameters to reach your goal:
 * __calories__
 * __n_genes__
 * __n_cromosomas__
 * __split_point__
 * __overload_underload__
 
 
 __Note:__ Is important not to exceed the amount of ingredients stored in 'file.csv' with the parameters **n_genes** * **n_cromosomas** in this case not greater than __1070__.
 
__Issues__: At the end of the training step, the recipes could be duplicated and therefore the 'n' recipes can be halved.

In [2]:
# Import Libraries 
import numpy as np
import pandas as pd

In [3]:
class Recipes_GAs:
    def __init__(self, calories, n_genes = 1000, n_cromosomas = 3, split_point = None, overload_underload = 2):
        """
            Genetic Algorithms that generate several recipes that fit with the amount of
            calories given.
            NOTE. The food is expected to be represented as 1 times portion grams 
            (1 * portion) grams.
                
                ** PARAMETERS:
                
                calories. [Float] Number of calories you expect in your recipe.
                n_genes.[Int] Number of recipes (rows or samples).
                n_cromosomas. [Int] Number of ingredients (columns).
                split_point. [Int] If you wanto to establish a static number to 
                                split the samples and create the new population.
                overload_underload. [Int] The number of samples to overload or underload,
                                            in each iteration if the best fitness is bigger
                                            than the calories, then we subtract 1 to 
                                            'n' random samples in a random column, on the other
                                            hand, if is smaller we add 1.
        """
        self.__calories = calories
        self.__porciones = self.__calories / 100
        self.__n_genes = n_genes
        self.__n_cromosomas = n_cromosomas
        self.__split_point = split_point
        self.__food = pd.read_csv("food.csv")
        self.__overload_underload = overload_underload
        self.__recipes = []
        self.__ite = 0
    
    """
        Method to get the best recipe or recipes, the number of printed recipes
        depends on the n_recipes parameter, by default 1.
    """
    def get_recipes(self, n_recipes = 1):
        food_type, food_amount, fitness = self.__recipes
        
        if n_recipes > len(food_type):
            n_recipes = len(food_type)
        print(f"Recetas Totales {food_amount.shape[0]}")
        print("Iteraciones \t", self.__ite, f" \t Porciones de {self.__porciones} gr\n")
        for i in range(n_recipes):
            recipe = min(range(len(fitness)), key= lambda i: abs(fitness[i] - self.__calories))
            print(f"Receta {i+1}: \n")
            print(f"Calorias: {fitness[recipe]}\n")
            for _type, amount in zip(food_type[recipe], food_amount[recipe]):
                print(f"* Porciones: {amount} \t Food: { self.__food['food'][_type] }")
            print("-------------------------\n")
            food_amount = np.delete(food_amount, recipe, axis= 0)
            food_type = np.delete(food_type, recipe, axis= 0)
            fitness = np.delete(fitness, recipe, axis= 0)
            
    """
        Method to generate population, it will return the assigned food (food_type) and
        the amount for each type (food_amount).
        The 'minum' and 'maximum' values determine the range of random portions that can be chosen.
    """
    def _generate_population(self):
        minimum = 1
        maximum = np.log10(self.__calories) + 1 - np.log10(self.__n_cromosomas)
        self.__food["calories"] = self.__food["calories"] * self.__porciones
        
        food_amount = np.random.randint(minimum, maximum, (self.__n_genes, self.__n_cromosomas))
        food_type = self.__food.sample(n= self.__n_genes * self.__n_cromosomas, replace= True).index.values.reshape(self.__n_genes, self.__n_cromosomas)
        return food_amount, food_type
        
    """
        This method calculate the total amount of calories based on the type of food and the 
        random number generated for it.
    """
    def _calculate_fitness(self, food_amount, food_type):
        food_calories = [ self.__food["calories"][ind] for ind in food_type ]
        fitness = [amount.dot(calories)  for amount, calories in zip(food_amount, food_calories)]
        return fitness
    
    """
        Roultte Selection, it will return two samples, moother and father.
        
    """
    def roulette_selection(self, fitness):
        total_fitness = np.sum(fitness)
        selected_sample = []
        for i in range(2):
            r = random.random()
            c = total_fitness * r
            Ca = 0
            for sample in fitness:
                Ca += sample
                if Ca >= c:
                    indx = fitness.index(sample)
                    selected_sample.append(indx)
                    break
        return selected_sample
    
    """
        Generate the new population, it will generate a new list for food amount (food_amoun) and 
        a new list for food type (food_type). 
        To select random samples we use the Roulette Selection and then we split it and create two
        new samples with them.
    """
    def generate_new_population(self, food_amount, food_type, fitness):
        new_population_amount = []
        new_population_type = []
        for i in range( int(self.__n_genes/2) ):
            selected_sample = self.roulette_selection(fitness)
            
            father_amount = food_amount[selected_sample[0]]
            father_type = food_type[selected_sample[0]]
            mother_amount = food_amount[selected_sample[1]]
            mother_type = food_type[selected_sample[1]]
            
            if(self.__split_point):
                split_p = self.__split_point
            else:
                split_p = random.randint(1, self.__n_genes - 2)
                
            child1_amount = np.concatenate([father_amount[:split_p], mother_amount[split_p:] ])
            child1_type = np.concatenate([father_type[:split_p], mother_type[split_p:]])
            child2_amount = np.concatenate([mother_amount[:split_p], father_amount[split_p:]])
            child2_type = np.concatenate([mother_type[:split_p], father_type[split_p:]])
            
            new_population_amount.append(child1_amount)
            new_population_amount.append(child2_amount)
            new_population_type.append(child1_type)
            new_population_type.append(child2_type)
            
        return new_population_amount, new_population_type

    """
        Method to solve overload or underload of the recipes, subtract 1 or add 1 to a random
        column of 'n' samples, the times this process is done is conditioned to
        the overload_underload parameter.
    """
    def _Overload_Underload(self, food_amount, bigger):
        for i in range(self.__overload_underload):
            indx_row = random.randint(0, self.__n_genes-1)
            indx_col = random.randint(0, self.__n_cromosomas-1)
            if(food_amount[indx_row][indx_col] > 1):
                if(bigger):
                    food_amount[indx_row][indx_col] -= 1
                else:
                    food_amount[indx_row][indx_col] += 1
        return food_amount
        
    
    """
        Fit method, here we implement all the methods of the class and
        start the training model.
    """
    def fit(self, epochs):
        food_amount, food_type = self._generate_population()
        ite = 0
        for ite in range(epochs):
            fitness = self._calculate_fitness(food_amount, food_type)
            
            """
                If fitness equal to calories parameter.
            """
            if self.__calories in set(fitness):
                break
                
            """ 
                If Fitness is bigger than calories parameter then apply overload
                method.
            """
            pivot = min(fitness, key= lambda i: abs(i - self.__calories))
            if(pivot > self.__calories):
                self._Overload_Underload(food_amount, True)
            else:
                self._Overload_Underload(food_amount, False)
            
            new_population_amount, new_population_type = self.generate_new_population(food_amount, food_type, fitness)
            food_amount = np.array(new_population_amount)
            food_type = np.array(new_population_type)
        
        # Drop duplicated recipes
        ind = pd.DataFrame(food_type)
        ind = ind[ ind.duplicated() ].index.values
        
        food_type = np.delete(food_type, ind, axis= 0)
        food_amount = np.delete(food_amount, ind, axis= 0)
        fitness = np.delete(fitness, ind, axis= 0)
        
        self.__recipes = [food_type, food_amount, fitness]
        self.__ite = ite
    

In [4]:
obj = Recipes_GAs(calories= 10000, n_genes= 1000, n_cromosomas= 5, overload_underload= 3)
obj.fit(500)

In [5]:
obj.get_recipes(5)

Recetas Totales 1000
Iteraciones 	 0  	 Porciones de 100.0 gr

Receta 1: 

Calorias: 10000.0

* Porciones: 1 	 Food: butter-margarine blend
* Porciones: 3 	 Food: rice flour
* Porciones: 3 	 Food: frijol largo
* Porciones: 3 	 Food: pepperoni
* Porciones: 3 	 Food: pescado blanco
-------------------------

Receta 2: 

Calorias: 9970.0

* Porciones: 3 	 Food: néctar de guanábana
* Porciones: 1 	 Food: buckwheat groats
* Porciones: 3 	 Food: tennis bread
* Porciones: 3 	 Food: bebida de jugo de naranja y albaricoque
* Porciones: 1 	 Food: verduras
-------------------------

Receta 3: 

Calorias: 10050.0

* Porciones: 3 	 Food: pez
* Porciones: 3 	 Food: little caesars 14" pepperoni pizza
* Porciones: 2 	 Food: millet
* Porciones: 2 	 Food: stagg country chili w/bns
* Porciones: 2 	 Food: sal
-------------------------

Receta 4: 

Calorias: 9950.0

* Porciones: 2 	 Food: salchicha de queso
* Porciones: 3 	 Food: braunschweiger (una salchicha de hígado)
* Porciones: 3 	 Food: morningstar f